## 7.7 Pytorch实现词性判别
我们知道每一个词都有词性，如train这个单词，可表示火车或训练等意思，具体表示为哪种词性，跟这个词所处的环境或上下文密切相关。根据上下文来确定词性是循环网络擅长的事，因为循环网络，尤其是LSTM或GRU网络，具有记忆功能。
	这节将使用LSTM网络实现词性判别。

### 7.7.1 词性判别主要步骤
	如何用LSTM对一句话里的各词进行词性标注？需要采用哪些步骤？这些问题就是这节将涉及的问题。用LSTM实现词性标注，我们可以采用以下步骤。
1.实现词的向量化。      
	假设有两个句子，作为训练数据，这两个句子的每个单词都已标好词性。当然我们不能直接把这两个语句直接输入LSTM模型，输入前需要把每个语句的单词向量化。假设这个句子共有5个单词，通过单词向量化后，就可得到序列[V_1, V_2, V_3, V_4, V_5],其中V_i表示第i个单词对应的向量。如何实现词的向量化？我们可以直接利用nn.Embedding层即可。当然在使用该层之前，需要把每句话对应单词或词性用整数表示。   
2.构建网络  
	词向量化之后，需要构建一个网络来训练，可以构建一个只有三层的网络，第一层为词嵌入层，第二层为LSTM层，最后一层用于词性分类的全连接层。
	下面用PyTorch实现这些步骤。

### 7.7.2 数据预处理

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms

In [2]:
#定义训练数据
training_data = [
    ("The cat ate the fish".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("They read that book".split(), ["NN", "V", "DET", "NN"])
]
#定义测试数据
testing_data=[("They ate the fish".split())]

In [3]:
testing_data

[['They', 'ate', 'the', 'fish']]

In [4]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

{'The': 0, 'cat': 1, 'ate': 2, 'the': 3, 'fish': 4, 'They': 5, 'read': 6, 'that': 7, 'book': 8}


In [5]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2} # 手工设定词性标签数据字典

### 7.7.3 构建网络
构建训练网络，共三层，分别为嵌入层、LSTM层、全连接层。

In [6]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.words_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        """
         初始化隐含状态State及C
        """
        return (torch.zeros(1,1, self.hidden_dim),
                torch.zeros(1,1, self.hidden_dim))

    def forward(self, sentence):
        # 获得词嵌入矩阵 embeds
        embeds = self.words_embeddings(sentence)
        # 按 lstm 格式， 修改 embeds 的形状
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        # 修改隐含状态的形状，作为全连接层的输入
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        # 计算每个单词属于各词性的概率
        tag_scores = F.log_softmax(tag_space, dim = 1)
        return tag_scores

其中有一个 nn.Embedding(vocab_size, embed_dim) 类，它是 Module 类的子类，这里它接收最重要的两个初始化参数：词汇量大小，每个词汇向量表示的向量维度。Embedding 类返回的是一个形状为[每句词个数，词维度]的矩阵。nn.LSTM 层的输入形状为（序列长度，批量大小，输入的大小），序列长度就是时间步序列长度，这个长度是可变的。F.log_softmax() 执行的是一个 Softmax 回归的对数。
	把数据转换为模型要求的格式，即把输入数据需要转换为 torch.LongTensor 张量。

In [7]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return tensor

### 7.7.4 训练网络

In [8]:
EMBEDDING_DIM = 10 
HIDDEN_DIM = 3 # 里等于词性个数

In [9]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(word_to_ix))

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
# print(training_data[0][0])
# print(inputs)
# print(model.words_embeddings(inputs))
# print(tag_scores)
# print(torch.max(tag_scores,1))

In [10]:
training_data[0][0]

['The', 'cat', 'ate', 'the', 'fish']

In [11]:
inputs

tensor([0, 1, 2, 3, 4])

In [12]:
model.words_embeddings(inputs)

tensor([[ 0.6143, -0.5330, -0.0048, -0.6938, -0.0964,  0.4655,  0.7679,  1.1049,
          0.8730, -1.6793],
        [ 0.9578, -0.7478, -0.3314, -0.5115, -0.1223, -0.8687, -1.5679, -0.6573,
          0.5399, -0.3160],
        [-1.1446, -1.4013, -0.2086,  2.2012, -0.1276, -1.9518, -1.0941, -2.7696,
         -1.3319, -0.5725],
        [ 1.4203,  1.0254, -0.5082,  0.8009, -2.3376, -0.1140,  0.1446,  0.8640,
          0.7840, -0.7120],
        [ 0.1141,  0.1341,  1.3729,  0.7346,  1.1318,  1.0677, -0.0249,  0.5255,
         -0.0466,  0.8431]], grad_fn=<EmbeddingBackward0>)

In [13]:
tag_scores

tensor([[-2.1873, -2.3365, -1.6993, -2.8101, -2.3794, -2.5123, -1.8858, -2.0785,
         -2.3270],
        [-2.0756, -2.5240, -1.7017, -2.9252, -2.2788, -2.2199, -1.8718, -2.1993,
         -2.4937],
        [-1.9935, -2.5036, -1.8384, -2.7779, -2.3209, -2.3301, -1.7089, -2.3614,
         -2.4087],
        [-2.0755, -2.4760, -1.7213, -2.9162, -2.3102, -2.3322, -1.8021, -2.2168,
         -2.4393],
        [-2.0834, -2.4517, -1.7516, -2.7906, -2.3211, -2.3297, -1.8439, -2.2044,
         -2.4029]], grad_fn=<LogSoftmaxBackward0>)

In [14]:
torch.max(tag_scores,1)

torch.return_types.max(
values=tensor([-1.6993, -1.7017, -1.7089, -1.7213, -1.7516], grad_fn=<MaxBackward0>),
indices=tensor([2, 2, 6, 2, 2]))

In [15]:
for epoch in range(400): # 我们要训练400次。
    for sentence, tags in training_data:

        # 清除网络先前的梯度值
        model.zero_grad()

        # 重新初始化隐藏层数据
        model.hidden = model.init_hidden()

        # 按网络要求的格式处理输入数据和真实标签数据
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        # 实例化模型
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        
        loss.backward()
        optimizer.step()


### 7.7.5 测试模型

In [16]:
# 查看模型训练的结果
test_inputs = prepare_sequence(testing_data[0], word_to_ix)
tag_scores01 = model(test_inputs)

In [18]:
testing_data[0]

['They', 'ate', 'the', 'fish']

In [19]:
model.words_embeddings(test_inputs)

tensor([[-0.8431,  0.3452, -0.4240,  0.1023, -0.7963,  0.7494, -0.1096,  0.0420,
          0.2716,  0.6799],
        [-1.1539, -1.4429, -0.2558,  2.3111, -0.1338, -2.0195, -1.0771, -2.8065,
         -1.3804, -0.6637],
        [ 1.5805,  1.1018, -0.4447,  0.7557, -2.5713, -0.1095,  0.0480,  0.8009,
          0.7784, -0.9003],
        [-0.2201,  0.4084,  1.1775,  0.3427,  1.3471,  1.2596,  0.1661,  0.5814,
          0.3140,  1.0856]], grad_fn=<EmbeddingBackward0>)

In [20]:
tag_scores01

tensor([[-6.3161, -0.0143, -4.6825, -7.7500, -7.7807, -8.2872, -7.5019, -7.1523,
         -7.2702],
        [-3.4784, -3.7046, -0.0675, -6.7686, -6.3365, -6.1042, -6.7601, -6.1223,
         -6.6671],
        [-0.0481, -4.2231, -3.7675, -7.0040, -6.3620, -6.2406, -6.1437, -6.7060,
         -6.7185],
        [-2.8125, -0.0972, -3.7817, -6.9446, -6.4851, -6.9245, -6.1296, -6.1145,
         -6.2541]], grad_fn=<LogSoftmaxBackward0>)

In [21]:
torch.max(tag_scores01,1)

torch.return_types.max(
values=tensor([-0.0143, -0.0675, -0.0481, -0.0972], grad_fn=<MaxBackward0>),
indices=tensor([1, 2, 0, 1]))